# 第3章: 正規表現
Wikipediaの記事を以下のフォーマットで書き出したファイル[jawiki-country.json.gz](/data/jawiki-country.json.gz)がある。

* 1行に1記事の情報がJSON形式で格納される
* 各行には記事名が"title"キーに、記事本文が"text"キーの辞書オブジェクトに格納され、そのオブジェクトがJSON形式で書き出される
* ファイル全体はgzipで圧縮される

以下の処理を行うプログラムを作成せよ。

## ※事前準備

### 必要なデータのダウンロード

In [ ]:
%%bash
FILE_NAME="chapter3/data/jawiki-country.json.gz"
URL="https://raw.githubusercontent.com/Mocchaso/nlp100knocks_2025/refs/heads/main/${FILE_NAME}"

mkdir -p $(dirname "${FILE_NAME}")
test -f "${FILE_NAME}" || curl -L "${URL}" -o "${FILE_NAME}"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 4949k  100 4949k    0     0  7973k      0 --:--:-- --:--:-- --:--:-- 7970k


## 20. JSONデータの読み込み
Wikipedia記事のJSONファイルを読み込み、「イギリス」に関する記事本文を表示せよ。問題21-29では、ここで抽出した記事本文に対して実行せよ。

In [ ]:
import pandas as pd

df = pd.read_json("chapter3/data/jawiki-country.json.gz", lines=True)
uk_article = df[df["title"] == "イギリス"]["text"].values[0]
# print(uk_article)

## 21. カテゴリ名を含む行を抽出
記事中でカテゴリ名を宣言している行を抽出せよ。

In [ ]:
import re

uk_records = uk_article.split("\n")

category_records = [
  uk_record for uk_record in uk_records
  if re.search(r"\[\[Category:.+\]\]", uk_record)
]

print(category_records)

['[[Category:イギリス|*]]', '[[Category:イギリス連邦加盟国]]', '[[Category:英連邦王国|*]]', '[[Category:G8加盟国]]', '[[Category:欧州連合加盟国|元]]', '[[Category:海洋国家]]', '[[Category:現存する君主国]]', '[[Category:島国]]', '[[Category:1801年に成立した国家・領域]]']


## 22. カテゴリ名の抽出
記事のカテゴリ名を（行単位ではなく名前で）抽出せよ。

In [ ]:
import re

category_names = [
    m.group(1)
    for category_record in category_records
    if (m := re.search(r"\[\[Category:([^|\]]+)", category_record))
]

print(category_names)

['イギリス', 'イギリス連邦加盟国', '英連邦王国', 'G8加盟国', '欧州連合加盟国', '海洋国家', '現存する君主国', '島国', '1801年に成立した国家・領域']


### memo

`([^|\]]+)` -> `|`, `]` 以外の任意の1文字を連続で得る。  
`|` か `]` が現れた時点でこのマッチングが進めなくなり、結果としてマッチがその位置で確定する。  
そのため、`|` 以降に現れる `*` の文字列をマッチングパターン内で別途記述する必要はない。

`[^...]` : **否定の文字クラス**。

---

`:=` : セイウチ演算子。  
式の中で変数への代入も一緒に行うことができる。  
Python 3.8 から導入された機能。

## 23. セクション構造
記事中に含まれるセクション名とそのレベル（例えば"== セクション名 =="なら1）を表示せよ。

In [ ]:
import re

sections = re.findall(r"^(=+)(.+)\1$", uk_article, re.MULTILINE)
for section in sections:
  print(f"level: {len(section[0]) - 1}, name: {section[1]}")

level: 1, name: 国名
level: 1, name: 歴史
level: 1, name: 地理
level: 2, name: 主要都市
level: 2, name: 気候
level: 1, name: 政治
level: 2, name: 元首
level: 2, name: 法
level: 2, name: 内政
level: 2, name: 地方行政区分
level: 2, name: 外交・軍事
level: 1, name: 経済
level: 2, name: 鉱業
level: 2, name: 農業
level: 2, name: 貿易
level: 2, name: 不動産
level: 2, name: エネルギー政策
level: 2, name: 通貨
level: 2, name: 企業
level: 3, name: 通信
level: 1, name: 交通
level: 2, name: 道路
level: 2, name: 鉄道
level: 2, name: 海運
level: 2, name: 航空
level: 1, name: 科学技術
level: 1, name: 国民
level: 2, name: 言語
level: 2, name: 宗教
level: 2, name: 婚姻
level: 2, name: 移住
level: 2, name: 教育
level: 2, name: 医療
level: 1, name: 文化
level: 2, name: 食文化
level: 2, name: 文学
level: 2, name: 哲学
level: 2, name: 音楽
level: 3, name: ポピュラー音楽
level: 2, name: 映画
level: 2, name: コメディ
level: 2, name: 国花
level: 2, name: 世界遺産
level: 2, name: 祝祭日
level: 2, name: スポーツ
level: 3, name: サッカー
level: 3, name: クリケット
level: 3, name: 競馬
level: 3, name: モータースポーツ
level: 3, name: 野球
level: 3, 

### memo

* `re.MULTILINE`: 文字列全体だけでなく、各行の先頭・末尾にマッチさせる。
* `\1`: 後方参照

---

[Wikipedia のセクション表記について](https://ja.wikipedia.org/wiki/Help:%E3%82%BB%E3%82%AF%E3%82%B7%E3%83%A7%E3%83%B3)

```
== セクションの見出し ==
=== サブセクションの見出し ===
==== サブサブセクションの見出し ====
```


## 24. ファイル参照の抽出
記事から参照されているメディアファイルをすべて抜き出せ。

In [ ]:
import re

media_files = re.findall(r"\[\[(ファイル|File):([^\]\|]+)(\|.*?)?\]\]", uk_article)
for i, media_file in enumerate(media_files, 1):
  print(f"{i}: {media_file[1]}")

1: Royal Coat of Arms of the United Kingdom.svg
2: United States Navy Band - God Save the Queen.ogg
3: Descriptio Prime Tabulae Europae.jpg
4: Lenepveu, Jeanne d'Arc au siège d'Orléans.jpg
5: London.bankofengland.arp.jpg
6: Battle of Waterloo 1815.PNG
7: Uk topo en.jpg
8: BenNevis2005.jpg
9: Population density UK 2011 census.png
10: 2019 Greenwich Peninsula & Canary Wharf.jpg
11: Birmingham Skyline from Edgbaston Cricket Ground crop.jpg
12: Leeds CBD at night.jpg
13: Glasgow and the Clyde from the air (geograph 4665720).jpg
14: Palace of Westminster, London - Feb 2007.jpg
15: Scotland Parliament Holyrood.jpg
16: Donald Trump and Theresa May (33998675310) (cropped).jpg
17: Soldiers Trooping the Colour, 16th June 2007.jpg
18: City of London skyline from London City Hall - Oct 2008.jpg
19: Oil platform in the North SeaPros.jpg
20: Eurostar at St Pancras Jan 2008.jpg
21: Heathrow Terminal 5C Iwelumo-1.jpg
22: Airbus A380-841 G-XLEB British Airways (10424102995).jpg
23: UKpop.svg
24: Anglos

### memo

[Wikipedia Help:画像などのファイルのアップロードと利用 - ファイルの埋め込み](https://ja.wikipedia.org/wiki/Help:%E7%94%BB%E5%83%8F%E3%81%AA%E3%81%A9%E3%81%AE%E3%83%95%E3%82%A1%E3%82%A4%E3%83%AB%E3%81%AE%E3%82%A2%E3%83%83%E3%83%97%E3%83%AD%E3%83%BC%E3%83%89%E3%81%A8%E5%88%A9%E7%94%A8#%E3%83%95%E3%82%A1%E3%82%A4%E3%83%AB%E3%81%AE%E5%9F%8B%E3%82%81%E8%BE%BC%E3%81%BF)

```
[[ファイル:tst.png]]
```

---

以下の全てのケースに対応させる必要がある。

* `[[ファイル:~~~]]` (パイプ無し)
* `[[ファイル:~~~|~~~]]` (パイプあり)
* 上記2種類の記述が同じ行内に2回以上現れるケース (パイプ部分も、1個のファイルの記述内で複数回現れるケースあり)

↓

**【各部分でやっていることの整理】**

* `\[\[` -> マークアップ開始
* `(ファイル|File):` -> ファイルの名前空間のプレフィックス
* `([^\]\|]+)` -> ファイル名そのもの  
`]` と `|` 以外の1文字を連続で取り出す。（ **否定の文字クラス** と繰り返し `+` の合わせ技）  
= `]` or `|` が出てきたら、ファイル名としては終わりと判断する。
* `(\|.*?)?` -> パイプ以降のオプション
  * `\|~~~` : パイプ部分の開始を検出する。
  * `.*?` : 任意の文字列を **最短一致** で抽出する。  
  最長一致 (`.*`) ではなく最短一致 (`.*?`) にすることで、  
  直後に `]]` が現れた時点でマッチングを止める。  
  = ファイルのマークアップが同じ行内に2回以上現れても、別々の塊としてマッチングできる。
  * `(...)?` で、グループ自体を任意扱いとする。
* `\]\]` -> マークアップ終了

※ [最長一致と最短一致](https://www.megasoft.co.jp/mifes/seiki/about.html)

正規表現で繰り返しの `*` や `+` など指定した場合、  
できるだけ長い文字列を得ようとするのが「最長一致」、  
できるだけ短い文字列を得ようとするのが「最短一致」。  
「最短一致」にしたいときは、繰り返しのメタ文字の後に `?` を付ける。

* 例: 文字列 `AAAAAAAAAA`
  * 最長一致 `A+` の場合 -> `AAAAAAAAAA` 全体がヒット
  * 最長一致 `A+?` の場合 -> 先頭の `A`のみヒット
* 例: 文字列 `<h1>タイトルタイトル</h1>`
  * 最長一致 `<.+>` の場合 -> `<h1>タイトルタイトル</h1>` 全体がヒット
  * 最短一致 `<.+?>` の場合 -> 先頭の `<h1>` のみヒット

## 25. テンプレートの抽出
記事中に含まれる「基礎情報」テンプレートのフィールド名と値を抽出し、辞書オブジェクトとして格納せよ。

In [ ]:
import re

# テンプレート部分を抽出
m = re.search(r"\{\{基礎情報 国\n(.*?)\n\}\}", uk_article, re.DOTALL)
template_body = m.group(1) if m else ""
# print("template_body:")
# print(template_body)
# print()
# 更にフィールド名と値を抽出
pairs = re.findall(
    r"^\|(.+?)\s*=\s*(.*?)(?=\n\||\n\}\})",
    template_body,
    re.MULTILINE | re.DOTALL
)
basic_info = {k.strip(): v.strip() for k, v in pairs}

basic_info

{'略名': 'イギリス',
 '日本語国名': 'グレートブリテン及び北アイルランド連合王国',
 '公式国名': '{{lang|en|United Kingdom of Great Britain and Northern Ireland}}<ref>英語以外での正式国名:<br />\n*{{lang|gd|An Rìoghachd Aonaichte na Breatainn Mhòr agus Eirinn mu Thuath}}（[[スコットランド・ゲール語]]）\n*{{lang|cy|Teyrnas Gyfunol Prydain Fawr a Gogledd Iwerddon}}（[[ウェールズ語]]）\n*{{lang|ga|Ríocht Aontaithe na Breataine Móire agus Tuaisceart na hÉireann}}（[[アイルランド語]]）\n*{{lang|kw|An Rywvaneth Unys a Vreten Veur hag Iwerdhon Glédh}}（[[コーンウォール語]]）\n*{{lang|sco|Unitit Kinrick o Great Breetain an Northren Ireland}}（[[スコットランド語]]）\n**{{lang|sco|Claught Kängrick o Docht Brätain an Norlin Airlann}}、{{lang|sco|Unitet Kängdom o Great Brittain an Norlin Airlann}}（アルスター・スコットランド語）</ref>',
 '国旗画像': 'Flag of the United Kingdom.svg',
 '国章画像': '[[ファイル:Royal Coat of Arms of the United Kingdom.svg|85px|イギリスの国章]]',
 '国章リンク': '（[[イギリスの国章|国章]]）',
 '標語': '{{lang|fr|[[Dieu et mon droit]]}}<br />（[[フランス語]]:[[Dieu et mon droit|神と我が権利]]）',
 '国歌': "[[女王陛下万歳|{{lang|en|God Save t

### memo

2つ目の正規表現について（フィールド名・値の抽出）

**【各部分でやっていることの整理】**

* `^` : `re.MULTILINE` により、各行の先頭がマッチするように制御。
* `\|` : フィールド開始記号。
* `(.+?)` : フィールド名。
* `\s*=\s*` : フィールド名・値の区切りの部分。
* `(.*?)` : 値部分。  
  * `re.DOTALL` により、値が複数行に跨っていてもマッチするように制御。
  * 次のフィールド・値のペアまでは巻き込まないようにするため、**最短一致** で取得。
* `(?=\n\||\n\}\})` : 終了条件。
  * `?=` : **肯定先読み**
  * `\n\|` (次のフィールド開始) か `\n\}\}` (テンプレート全体の末尾) が次に来る場合、  
  直前の値部分のマッチを止めるように制御する。

※ [先読み/後読みについて](https://qiita.com/shotets/items/98f3828b6e5f08d42498)

| 書き方 | 名前 | 意味 |
| - | - | - |
| `(?=pattern)` | 肯定先読み | 直後（読む先）にpatternがある |
| `(?!pattern)` | 否定先読み | 直後（読む先）にpatternが無い |
| `(?<=pattern)` | 肯定後読み | 直前（読む後）にpatternがある |
| `(?<!pattern)` | 否定後読み | 直前（読む後）にpatternが無い |

| 例 | 意味 | 結果 |
| - | - | - |
| `for(?=each)` | for の読む先に each がある（forの右にマッチ） | foreach の頭の for にはマッチし、for にはマッチしない |
| `for(?!each)` | for の読む先に each がない（forの右にマッチ） | foreach はマッチされず、for にはマッチする |
| `(?<=for)each` | each を読む後に for がある （eachの左にマッチ） | foreach のお尻の each にはマッチし、each にはマッチしない |
| `(?<!for)each` | each を読む後に for がない（eachの左にマッチ） | foreach はマッチされず、each にはマッチする |

※先読み・後読みはマッチ位置の条件を確認するだけで、文字列自体は消費しない。

## 26. 強調マークアップの除去
25の処理時に、テンプレートの値からMediaWikiの強調マークアップ（弱い強調、強調、強い強調のすべて）を除去してテキストに変換せよ（参考: [マークアップ早見表](http://ja.wikipedia.org/wiki/Help:%E6%97%A9%E8%A6%8B%E8%A1%A8)）。

## 27. 内部リンクの除去
26の処理に加えて、テンプレートの値からMediaWikiの内部リンクマークアップを除去し、テキストに変換せよ（参考: [マークアップ早見表](http://ja.wikipedia.org/wiki/Help:%E6%97%A9%E8%A6%8B%E8%A1%A8)）。

## 28. MediaWikiマークアップの除去
27の処理に加えて、テンプレートの値からMediaWikiマークアップを可能な限り除去し、国の基本情報を整形せよ。

## 29. 国旗画像のURLを取得する
テンプレートの内容を利用し、国旗画像のURLを取得せよ。（ヒント: [MediaWiki API](http://www.mediawiki.org/wiki/API:Main_page/ja)の[imageinfo](https://www.mediawiki.org/wiki/API:Imageinfo)を呼び出して、ファイル参照をURLに変換すればよい）